In [1]:
import os
from dotenv import load_dotenv
from tavily import TavilyClient
from openai import OpenAI
load_dotenv()


True

In [2]:
client = OpenAI()

In [3]:
system_prompt = """
당신은 고도로 숙련된 팩트체크 전문가입니다. 인터넷 기사나 소셜 미디어 포스팅에 포함된 주장을 철저히 분석하고, 신뢰할 수 있는 증거와 자료를 바탕으로 각 주장에 대한 진위를 검증하는 임무를 수행합니다. 다음의 지침을 엄격히 준수하세요:

사실 확인 대상 식별:

텍스트 내의 주요 사실, 주장, 통계 및 인용구를 면밀히 식별합니다.
주장별로 핵심 내용을 분리하여 분석합니다.
신뢰할 수 있는 출처 조사:

학술 연구, 정부 발표, 공신력 있는 뉴스 매체, 공식 기관의 자료 등 신뢰할 수 있는 다양한 출처를 활용하여 각 주장을 검증합니다.
가능한 경우, 여러 출처에서 동일한 정보를 교차 확인합니다.
객관적 분석 및 결과 분류:

각 주장에 대해 ‘확인됨’, ‘불확실함’, ‘거짓’ 등 명확한 분류를 제공합니다.
검증 과정에서 나타난 모순점이나 불일치를 명확하게 기술합니다.
출처 및 증거 명시:

검증에 사용한 모든 출처를 구체적으로 명시하고, 각 출처의 신뢰도와 관련 정보를 함께 제공합니다.
인용 시에는 출처를 정확히 밝히고, 출처의 최신성을 고려합니다.
추가 검토 및 권고:

검증 과정에서 추가 확인이 필요한 사항이나, 명확한 결론을 내리기 어려운 부분이 있다면 이에 대해 추가 조사를 권고합니다.
편향이나 오해의 소지가 있는 표현에 대해 객관적인 설명을 덧붙입니다.
보고서 작성:

전체 분석 과정을 명확하고 구조적으로 정리하여, 각 주장에 대한 평가와 결론을 일목요연하게 제시합니다.
분석 결과를 쉽게 이해할 수 있도록 표나 목록 등을 활용합니다.
항상 객관적이고 증거 기반의 접근을 유지하며, 개인적 의견이나 추측 없이 사실에 근거한 분석만을 제공하세요. 
"""


In [4]:
def generate_text(system_prompt, user_prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "keywords_schema",
                "schema": {
                    "type": "object",
                    "properties": {
                        "keywords": {
                            "description": "The keywords that are used to search for fact-checking",
                            "type": "array",
                            "items": {
                                "type": "string"
                            }
                        },
                        "additionalProperties": False
                    }
                }
            }
        }
    )
    return response.choices[0].message.content

In [5]:
generate_text(system_prompt, "지구는 평평하다")

'{"keywords":["지구 평평설","지구 둥글다","지구 형태 과학적 증거","지구의 형태 주장 검증","평평한 지구 이론 사실 확인"]}'

In [14]:
tavily_prompt = """
당신은 고도로 숙련된 팩트 체크 전문가입니다. 주어진 텍스트(인터넷 기사나 SNS 포스팅 등)에서 여러 주장이 포함되어 있을 때, 각 주장이나 논점을 세분화하고 이를 뒷받침할 근거를 찾기 위한 최적의 검색어를 추천하는 임무를 수행합니다. 아래 지침을 엄격히 준수하세요:

주장 및 논점 세분화:

텍스트 내에서 확인할 주요 주장, 통계, 인용, 사건 및 논점을 면밀히 식별합니다.
각 주장별 핵심 요소(예: 인물, 사건, 날짜, 장소, 수치 등)를 분리하여 목록화합니다.
근거 자료 탐색용 검색어 도출:

각 주장에 대해 신뢰할 수 있는 근거 자료(학술 연구, 정부 발표, 공신력 있는 뉴스, 공식 기관 자료 등)를 찾을 수 있도록 구체적인 키워드를 생성합니다.
해당 주장과 관련된 핵심 용어, 동의어, 사건명, 날짜, 장소 등을 포함하여 검색어의 정확도를 높입니다.
고급 검색 연산자 활용 제안:

검색 결과의 정밀도를 높이기 위해 큰따옴표(""), AND, OR, - 등의 고급 검색 연산자를 어떻게 활용할 수 있을지 구체적인 예시와 함께 제안합니다.
예를 들어, "주장내용" AND "출처명" 같은 형태의 검색어를 추천합니다.
최신 정보 및 신뢰도 고려:

최신 기사나 공식 자료를 우선적으로 찾을 수 있도록 '최신', '2025', '최근' 등의 단어를 포함한 검색어 옵션을 제공합니다.
여러 출처에서 동일한 정보를 확인할 수 있는 조합 검색어를 함께 제시합니다.
검색 전략 보고서 작성:

각 주제나 주장별로 추천 검색어 리스트를 제시하고, 각 검색어가 왜 효과적인지 간략한 설명을 덧붙입니다.
사용자에게 검색어 선택의 이유와 기대되는 검색 결과의 유형(예: 통계, 공식 발표, 학술 자료 등)을 명확히 안내합니다.

"""

In [38]:
keywords = generate_text(tavily_prompt, "한국의 지난 2024년 총선은 중국이 개입한 부정선거다.")
keywords


'{"keywords":["2024년 한국 총선","중국 개입 부정선거","한국 총선 부정선거","2024년 총선 결과","중국 정치 개입","한국 선거 부정행위","2024년 한국 총선 의혹","최근 한국 총선","부정선거 사례","중국의 한국 내 영향"]}'

In [5]:
tavily_client = TavilyClient(api_key=f"{os.getenv('TAVILY_API_KEY')}")

In [6]:
def search_tavily(query):
    response = tavily_client.search(
        query=query
    )
    return response


In [39]:
keywords

'{"keywords":["2024년 한국 총선","중국 개입 부정선거","한국 총선 부정선거","2024년 총선 결과","중국 정치 개입","한국 선거 부정행위","2024년 한국 총선 의혹","최근 한국 총선","부정선거 사례","중국의 한국 내 영향"]}'

In [40]:
import json
keywords = json.loads(keywords)

In [24]:
for keyword in keywords.values():
    print(keyword)

['2024 한국 총선 부정선거', '중국 개입 한국 선거', '한국 총선 외국 개입', '2024년 한국 총선 논란', '정치적 조작 한국 총선', '한국 총선 진실', '중국 정치 개입', '2024 한국 총선 결과 분석', '부정선거 증거 한국 총선', '중국 소셜 미디어 한국 총선']


In [41]:
keywords['keywords'] = keywords['keywords'][:3]

In [42]:
keywords

{'keywords': ['2024년 한국 총선', '중국 개입 부정선거', '한국 총선 부정선거']}

In [35]:
def process_data(keywords):
    results = {}
    for keywords in keywords.values():
        for keyword in keywords:
            results[keyword] = search_tavily(keyword)
    return results

In [43]:
results = process_data(keywords)
results

{'2024년 한국 총선': {'query': '2024년 한국 총선',
  'follow_up_questions': None,
  'answer': None,
  'images': [],
  'results': [{'title': '2024 총선 - Kbs 뉴스',
    'url': 'https://news.kbs.co.kr/special/election2024/main.html',
    'content': '2024 총선 | 제 22대 국회의원 선거와 관련된 정보를 빠르고 정확하게 알려드립니다. 여론조사, 선거뉴스, 선거영상 등 다양한 콘텐츠를 kbs 총선 특집사이트에서 확인하세요. ... 4년 (2024.5.30. ~ 2028.5.29.) 재·보궐선거는 전임자의 잔여 임기',
    'score': 0.7610953,
    'raw_content': None},
   {'title': '대한민국 제22대 국회의원 선거 - 위키백과, 우리 모두의 백과사전',
    'url': 'https://ko.wikipedia.org/wiki/대한민국_제22대_국회의원_선거',
    'content': '대한민국 제22대 국회의원 선거는 2024년 4월 10일에 실시된 대한민국의 국회의원 선거이다. 지역구 국회의원 254명, 비례대표 국회의원 46명을 선출하였다. 국회의원 총선거와 함께 2024년 4월 대한민국 재보궐선거도 동시에 치러졌다.. 이 선거는 윤석열 정부 집권 3년차에 접어들기 직전에 치러지는',
    'score': 0.69672287,
    'raw_content': None},
   {'title': '2024 총선: 집권당 역대급 참패 원인과 앞으로의 전망 - Bbc',
    'url': 'https://www.bbc.com/korean/articles/crgyxz146x3o',
    'content': '2024 총선; 한국; 관련 기사 더 보기 ... 2024년 4월 5일. 22대 총선 개표 결과 보기 2024년 4월 11일',
 

In [48]:
for result in results.values():
    print(result)

{'query': '2024년 한국 총선', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '2024 총선 - Kbs 뉴스', 'url': 'https://news.kbs.co.kr/special/election2024/main.html', 'content': '2024 총선 | 제 22대 국회의원 선거와 관련된 정보를 빠르고 정확하게 알려드립니다. 여론조사, 선거뉴스, 선거영상 등 다양한 콘텐츠를 kbs 총선 특집사이트에서 확인하세요. ... 4년 (2024.5.30. ~ 2028.5.29.) 재·보궐선거는 전임자의 잔여 임기', 'score': 0.7610953, 'raw_content': None}, {'title': '대한민국 제22대 국회의원 선거 - 위키백과, 우리 모두의 백과사전', 'url': 'https://ko.wikipedia.org/wiki/대한민국_제22대_국회의원_선거', 'content': '대한민국 제22대 국회의원 선거는 2024년 4월 10일에 실시된 대한민국의 국회의원 선거이다. 지역구 국회의원 254명, 비례대표 국회의원 46명을 선출하였다. 국회의원 총선거와 함께 2024년 4월 대한민국 재보궐선거도 동시에 치러졌다.. 이 선거는 윤석열 정부 집권 3년차에 접어들기 직전에 치러지는', 'score': 0.69672287, 'raw_content': None}, {'title': '2024 총선: 집권당 역대급 참패 원인과 앞으로의 전망 - Bbc', 'url': 'https://www.bbc.com/korean/articles/crgyxz146x3o', 'content': '2024 총선; 한국; 관련 기사 더 보기 ... 2024년 4월 5일. 22대 총선 개표 결과 보기 2024년 4월 11일', 'score': 0.6867278, 'raw_content': None}, {'title': '2024년 한국 총선결과 - Kb